In [ ]:
from Bio import SeqIO

for seq_record in SeqIO.parse("ls_orchid.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))


    # output
    # id
    # length

gi|2765658|emb|Z78533.1|CIZ78533
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC')
740
gi|2765657|emb|Z78532.1|CCZ78532
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC')
753
gi|2765656|emb|Z78531.1|CFZ78531
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA')
748
gi|2765655|emb|Z78530.1|CMZ78530
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT')
744
gi|2765654|emb|Z78529.1|CLZ78529
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA')
733
gi|2765652|emb|Z78527.1|CYZ78527
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC')
718
gi|2765651|emb|Z78526.1|CGZ78526
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT')
730
gi|2765650|emb|Z78525.1|CAZ78525
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA')
704
gi|2765649|emb|Z78524.1|CFZ78524
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC')
740
gi|2765648|emb|Z78523.1|CHZ78523
Seq('CGTAACCAGGTTTCCGT

In [ ]:
# identifiers = [seq_record.id fpr seq_record in SeqIO.parse(ls_orc)]

In [6]:
from Bio  import SeqIO
record_iterator = SeqIO.parse("ls_orchid.fasta", "fasta")
first_record = next(record_iterator)
print(first_record.id)
print(first_record.description)

second_record = next(record_iterator)
print(second_record.id)
print(second_record.description)

gi|2765658|emb|Z78533.1|CIZ78533
gi|2765658|emb|Z78533.1|CIZ78533 C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA
gi|2765657|emb|Z78532.1|CCZ78532
gi|2765657|emb|Z78532.1|CCZ78532 C.californicum 5.8S rRNA gene and ITS1 and ITS2 DNA


In [7]:
first_record = next(SeqIO.parse("ls_orchid.gbk", "genbank"))
print(first_record.id)

Z78533.1


In [10]:
from Bio  import SeqIO
records = list(SeqIO.parse("ls_orchid.gbk", "genbank"))
print("found %i records" % len(records))
print("the last record")


found 94 records
the last record


In [11]:
print(first_record.annotations)

{'molecule_type': 'DNA', 'topology': 'linear', 'data_file_division': 'PLN', 'date': '30-NOV-2006', 'accessions': ['Z78533'], 'sequence_version': 1, 'gi': '2765658', 'keywords': ['5.8S ribosomal RNA', '5.8S rRNA gene', 'internal transcribed spacer', 'ITS1', 'ITS2'], 'source': 'Cypripedium irapeanum', 'organism': 'Cypripedium irapeanum', 'taxonomy': ['Eukaryota', 'Viridiplantae', 'Streptophyta', 'Embryophyta', 'Tracheophyta', 'Spermatophyta', 'Magnoliophyta', 'Liliopsida', 'Asparagales', 'Orchidaceae', 'Cypripedioideae', 'Cypripedium'], 'references': [Reference(title='Phylogenetics of the slipper orchids (Cypripedioideae: Orchidaceae): nuclear rDNA ITS sequences', ...), Reference(title='Direct Submission', ...)]}


In [1]:
from Bio import Align
from Bio.Align import substitution_matrices
import numpy as np

def smith_waterman_local_alignment(seq1, seq2, match_score=2, mismatch_score=-1, gap_penalty=-1):
    """
    Custom implementation of Smith-Waterman algorithm for local alignment
    """
    # Initialize scoring matrix
    m, n = len(seq1), len(seq2)
    score_matrix = np.zeros((m + 1, n + 1))
    traceback_matrix = np.zeros((m + 1, n + 1))
    
    # Fill the scoring matrix
    max_score = 0
    max_pos = (0, 0)
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Calculate match/mismatch score
            if seq1[i-1] == seq2[j-1]:
                diagonal_score = score_matrix[i-1][j-1] + match_score
            else:
                diagonal_score = score_matrix[i-1][j-1] + mismatch_score
            
            # Calculate gap scores
            up_score = score_matrix[i-1][j] + gap_penalty
            left_score = score_matrix[i][j-1] + gap_penalty
            
            # Take maximum (0 ensures local alignment)
            score_matrix[i][j] = max(0, diagonal_score, up_score, left_score)
            
            # Track max score position
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)
    
    # Backtracking to find alignment
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_pos
    
    while i > 0 and j > 0 and score_matrix[i][j] > 0:
        current_score = score_matrix[i][j]
        diagonal_score = score_matrix[i-1][j-1]
        up_score = score_matrix[i-1][j]
        left_score = score_matrix[i][j-1]
        
        if current_score == diagonal_score + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score):
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif current_score == up_score + gap_penalty:
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            i -= 1
        elif current_score == left_score + gap_penalty:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            j -= 1
        else:
            break
    
    # Reverse the aligned sequences
    aligned_seq1 = ''.join(reversed(aligned_seq1))
    aligned_seq2 = ''.join(reversed(aligned_seq2))
    
    return max_score, aligned_seq1, aligned_seq2, score_matrix

# Using Biopython's built-in aligner
def align_with_biopython(seq1, seq2):
    """
    Use Biopython's PairwiseAligner for Smith-Waterman alignment
    """
    # Create aligner object
    aligner = Align.PairwiseAligner()
    
    # Set alignment mode to local (Smith-Waterman)
    aligner.mode = 'local'
    
    # Set scoring parameters
    aligner.match_score = 2.0
    aligner.mismatch_score = -1.0
    aligner.open_gap_score = -1.0
    aligner.extend_gap_score = -1.0
    
    # Perform alignment
    alignments = aligner.align(seq1, seq2)
    
    # Get the best alignment
    best_alignment = next(alignments)
    
    return best_alignment

# Main execution
if __name__ == "__main__":
    # Given sequences
    sequence_X = "MAGNIFICENT"
    sequence_Y = "MAGNIFIC"
    
    print("=" * 60)
    print("SMITH-WATERMAN LOCAL ALIGNMENT")
    print("=" * 60)
    print(f"Sequence X: {sequence_X}")
    print(f"Sequence Y: {sequence_Y}")
    print("-" * 60)
    
    # Method 1: Custom implementation
    print("\n1. CUSTOM IMPLEMENTATION RESULTS:")
    score, aligned1, aligned2, matrix = smith_waterman_local_alignment(
        sequence_X, sequence_Y, 
        match_score=2, mismatch_score=-1, gap_penalty=-1
    )
    
    print(f"Best local alignment score: {score}")
    print(f"Aligned subsequence 1: {aligned1}")
    print(f"Aligned subsequence 2: {aligned2}")
    
    # Visual representation
    print("\nAlignment visualization:")
    print(f"X: {aligned1}")
    print(f"   {'|' * len([i for i, j in zip(aligned1, aligned2) if i == j])}")
    print(f"Y: {aligned2}")
    
    # Method 2: Using Biopython's built-in aligner
    print("\n" + "-" * 60)
    print("2. BIOPYTHON BUILT-IN ALIGNER RESULTS:")
    best_alignment = align_with_biopython(sequence_X, sequence_Y)
    
    print(f"Alignment score: {best_alignment.score}")
    print(f"Aligned sequences:\n{best_alignment}")
    
    # Show scoring matrix (first few rows/columns)
    print("\n" + "-" * 60)
    print("SCORING MATRIX (first few positions):")
    print("     " + "  ".join(sequence_Y[:8]))
    for i in range(min(9, len(sequence_X) + 1)):
        row = sequence_X[i-1] if i > 0 else ' '
        print(f"{row:2} " + " ".join(f"{int(matrix[i][j]):3}" for j in range(min(9, len(sequence_Y) + 1))))

SMITH-WATERMAN LOCAL ALIGNMENT
Sequence X: MAGNIFICENT
Sequence Y: MAGNIFIC
------------------------------------------------------------

1. CUSTOM IMPLEMENTATION RESULTS:
Best local alignment score: 16.0
Aligned subsequence 1: MAGNIFIC
Aligned subsequence 2: MAGNIFIC

Alignment visualization:
X: MAGNIFIC
   ||||||||
Y: MAGNIFIC

------------------------------------------------------------
2. BIOPYTHON BUILT-IN ALIGNER RESULTS:
Alignment score: 16.0
Aligned sequences:
target            0 MAGNIFIC 8
                  0 |||||||| 8
query             0 MAGNIFIC 8


------------------------------------------------------------
SCORING MATRIX (first few positions):
     M  A  G  N  I  F  I  C
     0   0   0   0   0   0   0   0   0
M    0   2   1   0   0   0   0   0   0
A    0   1   4   3   2   1   0   0   0
G    0   0   3   6   5   4   3   2   1
N    0   0   2   5   8   7   6   5   4
I    0   0   1   4   7  10   9   8   7
F    0   0   0   3   6   9  12  11  10
I    0   0   0   2   5   8  11  